#### Persionalize 구성 흐름
user interaction 최소 조건
- At minimum 1000 interactions records from users interacting with items in your catalog.
- At minimum 25 unique user IDs with at least 2 interactions for each.

1. 데이터 탐색
2. 데이터 정제
3. S3 Bucket 생성, Policy 설정,  정제 데이터 업로드


In [1]:
import pandas as pd
import datetime
pd.set_option('display.width', 1000)

In [2]:
data_df = pd.read_csv('./data/data.csv')
data_df.head()

/var/folders/8_/shv0pqd52xb1xxjjxsp4yptm0000gp/T/ipykernel_2526/322699375.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv('./data/data.csv')


,Unnamed: 0,event_time,event_type,product_id,brand,price,user_id,user_session,target,cat_0,cat_1,cat_2,cat_3
0,0,2022-11-01 00:00:14 UTC,cart,1005014,samsung,503.09,533326659,6b928be2-2bce-4640-8296-0efdf2fda22a,0,electronics,smartphone,NaN,NaN
1,1,2022-11-01 00:03:39 UTC,cart,1005115,apple,949.47,565865924,fd4bd6d4-bd14-4fdc-9aff-bd41a594f82e,0,electronics,smartphone,NaN,NaN
2,2,2022-11-01 00:05:54 UTC,cart,1002542,apple,486.80,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electronics,smartphone,NaN,NaN
3,3,2022-11-01 00:07:22 UTC,cart,1002542,apple,486.80,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electronics,smartphone,NaN,NaN
4,4,2022-11-01 00:10:45 UTC,cart,4804056,apple,160.57,522355747,0a1f37d1-71b7-4645-a8a7-ab91bc198a51,0,electronics,audio,headphone,NaN


In [3]:
print(data_df.shape)

(11495242, 13)


In [4]:
# instraction datasetd에 필수 항목 추출
insteractions_df = data_df[['user_id', 'product_id','event_time', 'event_type']]
# items_df = data_df[['product_id', 'price', 'brand', 'cat_0', 'cat_1', 'cat-2']]
print(insteractions_df.shape)
insteractions_df.isnull().sum()

(11495242, 4)


user_id       0
product_id    0
event_time    0
event_type    0
dtype: int64

In [5]:
# dataframe 내에 colname 이 multi_cnt 이상 df 반환
def filter_df(df, colname='user_id',  multi_cnt=2):
    # colname user_id 인 경우  아래 조건을 맞족한는 Series의 index 가 user_id
    is_multi = df[colname].value_counts() > 1
    # Series boolean index를 이용
    multi_index = is_multi[is_multi].index
    filtered = df[df[colname].isin(multi_index)]
    return filtered, multi_index

In [6]:
#  최소 거래건수: 2개 이상
filtered_df, user_ids_atleast_2event = filter_df(insteractions_df)
print(filtered_df.shape)


(10541336, 4)


In [7]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10541336 entries, 0 to 11495241
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user_id     int64 
 1   product_id  int64 
 2   event_time  object
 3   event_type  object
dtypes: int64(2), object(2)
memory usage: 402.1+ MB


- event_time:  object --> Unix timestamp 타입 변환 필요

### evnet_time type conversion: object to Unix timestamp

In [10]:
user_interactions_df = filtered_df.copy()

# 유효하지 않는  format은  NaT으로 변환 및 제거: object -> datetime
user_interactions_df['event_time'] = pd.to_datetime(user_interactions_df['event_time'], format="%Y-%m-%d %H:%M:%S UTC", errors='coerce')
user_interactions_df = user_interactions_df[pd.notnull(user_interactions_df['event_time'])]

In [11]:

# event_time의 datetime을 Unix timestamp 타입으로
user_interactions_df['event_time'] = user_interactions_df.event_time.apply(lambda x : (x-datetime.datetime(1970,1,1)).total_seconds())
user_interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10505424 entries, 0 to 11495241
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     int64  
 1   product_id  int64  
 2   event_time  float64
 3   event_type  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 400.8+ MB


In [12]:
# convert event_time from float to int
user_interactions_df['event_time'] = user_interactions_df['event_time'].astype(int)

In [13]:
user_interactions_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10505424 entries, 0 to 11495241
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user_id     int64 
 1   product_id  int64 
 2   event_time  int64 
 3   event_type  object
dtypes: int64(3), object(1)
memory usage: 400.8+ MB


In [14]:
# rename columne:  Personalize를 위한 필수 항목: USER_ID, ITEM_ID, TIMESTAMP
user_interactions_df.rename(columns={"user_id": "USER_ID", "product_id": "ITEM_ID", "event_time": "TIMESTAMP", "event_type": "EVENT_TYPE" }, inplace=True)


In [15]:
user_interactions_df.isnull().sum()

USER_ID       0
ITEM_ID       0
TIMESTAMP     0
EVENT_TYPE    0
dtype: int64

In [16]:
# 정제된 데이터를 파일로 저장
user_interactions_df.to_csv("./data/cleaned_user_interactions.csv", index=False)